- Reference: https://tableau.github.io/server-client-python/docs/api-ref
- "HDC현대산업개발": "2a21240d-3f53-4f02-944d-b01647c3dd51"

In [1]:
import tableauserverclient as TSC
import pandas as pd
import numpy as np
import pymssql
from dateutil.relativedelta import relativedelta

f = open("D:/Github/Work/Tableau/datamart_password.txt", "r")
# f = open("C:/Work/Tableau/datamart_password.txt", "r")
password = f.readline()

pd.set_option("display.max_columns", None)

In [2]:
# conn = pymssql.connect(server="133.186.215.165", database="HDCMART", user="hdcmart_user", password=password, charset="UTF8")
# query = """
# SELECT id_sabun, ds_hname, ds_order1, ds_order2, ds_dept, ds_position, ds_emptype, dt_order, dt_orderend 
# FROM DATAMART_DAAV_ORDER
# WHERE cd_corp = 'A101' AND dt_order > '20190101'
# """
# hist = pd.read_sql(query, conn)
# hist.columns = hist.columns.str.lower()
# for col in hist.columns:
#     if pd.api.types.is_string_dtype(hist[col]):
#         hist[col] = hist[col].str.encode("latin-1").str.decode("euc-kr")

In [3]:
hr = pd.read_excel("D:/Github/Work/Tableau/직원 명단_조직 기준_210930.xlsx")
hr["성명"] = hr["성명"].apply(lambda x:x.replace(" ", ""))
hr = hr[["사번", "EMAIL", "본부명(조직단위기준)", "조직명", "하위그룹", "직무", "직급", "성명"]]
hr = hr.rename({"사번":"num", "EMAIL":"email", "본부명(조직단위기준)":"hq", "조직명":"dep", "하위그룹":"sub", "직무":"job", "직급":"rank", "성명":"name"}, axis=1)
hr = hr.sort_values(by=["job", "rank", "hq", "dep"], axis=0)
hr["num"] = hr["num"].map(str)

hr.loc[hr["num"]=="1", "num"] = "0001"

empls = {str(row["num"]):(row["hq"], row["dep"], row["job"], row["name"]) for _, row in hr.iterrows()}

req_opts = TSC.RequestOptions(pagesize=1000)

# SIgn In

In [4]:
tableau_auth = TSC.TableauAuth("6363", "6363!hdc")
server = TSC.Server("https://datalab.hdc-dvp.com", use_server_version=True)

# Create Fundamental Datasets

In [109]:
# # user_group.csv
# user_group = list()
# with server.auth.sign_in(tableau_auth):
#     users, pagination_item = server.users.get(req_options=req_opts)
#     for user in users:
#         pagination_item = server.users.populate_groups(user, req_options=req_opts)
#         user_group.extend([(user.id, user.name, user.fullname, user.site_role) + empls[user.name] + (group.name,) if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, group.name) for group in user.groups if group.name != "All Users"])
# user_group = pd.DataFrame(user_group, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dep", "job", "name", "group"])

# user_group.to_csv("D:/Github/Work/Tableau/user_group.csv", index=False, encoding="euc-kr")

# # group_user.csv
# group_user = list()
# with server.auth.sign_in(tableau_auth):
#     groups, pagination_item = server.groups.get(req_options=req_opts)
#     for group in groups[1:]:
#         pagination_item = server.groups.populate_users(group, req_options=req_opts)
#         group_user.extend([(group.name,) + empls[user.name] + (user.site_role,) if user.name in empls.keys() else (group.name, None, None, None, user.name, user.site_role) for user in group.users])

# group_user = pd.DataFrame(group_user, columns=["group", "hq", "dep", "job", "name", "site_role"])

# group_user.to_csv("D:/Github/Work/Tableau/group_user.csv", index=False, encoding="euc-kr")

# # user_workbook
# user_wb = list()
# with server.auth.sign_in(tableau_auth):
#     users, pagination_item = server.users.get(req_options=req_opts)
#     for user in users:
#         pagination_item = server.users.populate_workbooks(user, req_options=req_opts)
#         user_wb.extend([(user.id, user.name, user.fullname, user.site_role) + empls[user.name] + (wb.name,) if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None, wb.name) for wb in user.workbooks if wb.project_name=="HDC현대산업개발"])
# user_wb = pd.DataFrame(user_wb, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dep", "job", "name", "wb"])

# user_wb.to_csv("D:/Github/Work/Tableau/user_wb.csv", index=False, encoding="euc-kr")

# # have_account
# have_account = hr[["num", "hq", "dep", "name", "sub", "job", "rank", "email"]]
# have_account["have"] = have_account["num"].apply(lambda x:True if str(x) in all_users["user_name"].tolist() else False)
# have_account.to_csv("D:/Github/Work/Tableau/have_account.csv", index=False, encoding="euc-kr")

# print("All completed!")

All completed!


C:\Users\00006363\Anaconda3\envs\tf2.3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Create Users

In [7]:
all_users[all_users["user_name"]=="5947"]

,user_id,user_name,user_fnm,site_role,hq,dep,job,name
2,63721ecb-87f7-4565-83a2-94a12c583fd3,5947,하지선(디지털플랫폼팀),ServerAdministrator,미래혁신본부,디지털플랫폼팀,팀원,하지선


In [5]:
users_to_create = hr[(hr["job"]=="회장") | (hr["job"]=="부회장") | (hr["job"]=="대표이사사장") | (hr["job"]=="본부장") | ((hr["sub"] != "정보기술외주") & (hr["job"]=="팀장")) | (hr["job"]=="실장") | (hr["job"]=="그룹장") | (hr["job"]=="부문장") | (hr["job"]=="단장") | (hr["job"]=="리더") | (hr["job"]=="PM") | (hr["job"]=="건축PM") | (hr["job"]=="TFT팀장") | (hr["job"]=="소장") | (hr["num"].isin(["4100435", "5720", "5704", "5707", "6084", "6161", "4001341", "4001360", "4001386", "5934", "4001380", "6119", "5218", "6110", "4632", "5308", "5104", "5882", "5892", "6168", "6335", "6353", "6284", "6337", "6281", "6427", "6403", "1101291", "6012", "5653", "6032", "6294", "6111", "5555", "5653", "5835", "5898", "6331", "6119", "6262", "6283", "4100222"]))]

with server.auth.sign_in(tableau_auth):
    for _, row in users_to_create.iterrows():
        user = TSC.UserItem(row["num"], "Viewer")
        try:
            user = server.users.add(user)
            if row["job"] in ["회장", "부회장", "대표이사사장", "본부장", "팀장", "실장", "그룹장", "부문장", "단장", "리더", "PM", "건축PM", "소장", "원가관리자"]:
                user.fullname = f"{row['name']}({row['job']})"
            else:
                user.fullname = f"{row['name']}({row['dep']})"
            user.email = row["email"]
            user = server.users.update(user, password=f"secretpassword")
        except:
            continue
            
# all_users.csv
with server.auth.sign_in(tableau_auth):
    all_users, pagination_item = server.users.get(req_options=req_opts)
all_users = [(user.id, user.name, user.fullname, user.site_role) + empls[user.name] if user.name in empls.keys() else (user.id, user.name, user.fullname, user.site_role, None, None, None, None) for user in all_users]
all_users = pd.DataFrame(all_users, columns=["user_id", "user_name", "user_fnm", "site_role", "hq", "dep", "job", "name"])

all_users.to_csv("D:/Github/Work/Tableau/all_users.csv", index=False, encoding="euc-kr")
            
print("All Completed!")

All Completed!


# Add Users to Groups

In [6]:
# all_groups.csv
with server.auth.sign_in(tableau_auth):
    all_groups, pagination_item = server.groups.get(req_options=req_opts)
all_groups = [(group.name, group.id) for group in all_groups]
all_groups = pd.DataFrame(all_groups, columns=["group", "group_id"])

all_groups.to_csv("D:/Github/Work/Tableau/all_groups.csv", index=False, encoding="euc-kr")

# 회장/부회장/대표이사사장/본부장
users_to_alloc = hr[(hr["job"]=="회장") | (hr["job"]=="부회장") | (hr["job"]=="대표이사사장") | (hr["job"]=="본부장") |((hr["job"]=="팀장") & (hr["dep"]=="디지털플랫폼팀")) | (hr["num"].isin(["6032", "6294", "6119", "6111", "6403", "1101291", "5634"]))]
tar_grp = "f827e0fe-ff32-4443-9c40-701c94a13ba1"

with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users.loc[all_users["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue

# 팀장/실장/그룹장/부문장/단장/리더/PM/건축PM
users_to_alloc = hr[((hr["sub"] != "정보기술외주") & (hr["job"]=="팀장")) | (hr["job"]=="실장") | (hr["job"]=="그룹장") | (hr["job"]=="부문장") | (hr["job"]=="단장") | (hr["job"]=="리더") | (hr["job"]=="PM") | (hr["job"]=="건축PM") | (hr["job"]=="TFT팀장")]
tar_grp = "39dc2d08-d291-4534-97d5-b1a6b3b6f67c"

with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users.loc[all_users["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue
        
# 소장
users_to_alloc = hr[(hr["job"]=="소장")]
tar_grp = "05fa4bd0-e1df-4971-bb89-4e39f24782d7"

with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users.loc[all_users["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue
            

# 본부장 회의
users_to_alloc = hr[(hr["num"].isin(["6427", "6168", "6335", "5704", "5720", "5882", "4001380", "4100435", "6281", "4001341", "4001360", "4001386", "5653", "6012"]))]
tar_grp = "0ef096dd-2c99-4473-b80b-855d1c0b29e1"

with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users.loc[all_users["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue
            
# 경영 현황 일반 사용자
tar_grp = "91d8104c-e669-4ea2-a5fa-27b6be938858"

# HR 현황 일반 사용자
users_to_alloc = hr[(hr["num"].isin(["6119", "5555", "5653", "5835", "5898", "6331", "6119", "6262", "6283", "4100222"]))]
tar_grp = "1ac1f9c5-d4b9-47d9-8c40-618df8f87f6b"

with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users.loc[all_users["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue
            
# 영업 현황 일반 사용자
tar_grp = "0715f81d-9385-4fc8-8c2a-76f8b9481dac"

# 현장 현황 일반 사용자
users_to_alloc = hr[(hr["num"].isin(["5947", "6353"]))]
tar_grp = "fcace1dd-cfa8-4f64-8b20-803c739e1e3c"

with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users.loc[all_users["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue

# 분양 상세 현황 일반 사용자
users_to_alloc = hr[(hr["num"].isin(["4001380", "5218", "6337", "4632"]))]
tar_grp = "3babc0fe-dbbb-403c-9577-27fe942bcf38"

with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users.loc[all_users["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue
            
# 관리자
users_to_alloc = hr[(hr["num"].isin(["6363", "5899", "5947"]))]
tar_grp = "6e2fc4f0-46f2-4114-8b25-341b38af9f7d"

with server.auth.sign_in(tableau_auth):
    groups, pagination_item = server.groups.get(req_options=req_opts)
    tar_grp = [i for i in groups if i.id == tar_grp][0]
    for _, row in users_to_alloc.iterrows():
        user_id = all_users.loc[all_users["user_name"]==row["num"], "user_id"].tolist()[0]
        try:
            server.groups.add_user(tar_grp, user_id)
        except:
            continue
            
print("All completed!")

All completed!


In [20]:
# # user_workbook
# with server.auth.sign_in(tableau_auth):
#     user = server.users.get_by_id("b71d75c3-a34c-4dc5-a92b-de46dc681212")
#     server.users.populate_workbooks(user)
#     for workbook in user.workbooks:
#         if workbook.project_name=="HDC현대산업개발":
#             print(workbook.name)

# Modify User Information

In [8]:
with server.auth.sign_in(tableau_auth):
    user = server.users.get_by_id("63721ecb-87f7-4565-83a2-94a12c583fd3")
#     user.fullname = "최혁(팀장)"
    user = server.users.update(user, password="secretpassword")
print("All Completed!")

All Completed!


# Remove Users

In [74]:
user_id = "c63d33fe-e955-4765-94d4-f83f2001b1c0"
with server.auth.sign_in(tableau_auth):
    server.users.remove(user_id)
print("Completed!")

Completed!
